# Training Run - Seasonal Prior & phase shifted inputs

#### Importing Libraries:

In [10]:
import subprocess
import os
import sys
import argparse
import shutil
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap.umap_ as umap
import os

from utils import *

### Config

In [ ]:
# define constants
INPUT_SIZE = 64*24
DEGREE = 3
LATENT_SIZE = 4*24
gnrt_start = '1970-01-01 00:00:00'
gnrt_end = '2020-12-31 16:00:00'

# Parameters
input_shape = INPUT_SIZE
latent_dim = None
latent_filter = 20
interim_filters = 20

# training hyperparameters
learning_rate = 0.001
epochs = 500
batch_size = 32

prior_dist_type = 'Seasonal'
activate_phase_shift = True

os.makedirs('results', exist_ok=True)
results_directory = f'results/training_run_{prior_dist_type}_prior_{activate_phase_shift}_phase_shift_{epochs}_epochs'

### Data Processing 

In [12]:
%matplotlib notebook

os.makedirs('data/processed', exist_ok=True)
os.makedirs('images', exist_ok=True)

set_seed(42)
df_list = []
for year in range(1970, 2021, 3):
    ds = xr.open_dataset(f'data/raw/phoenix/t2m_{year}_{year+2}/data_stream-oper_stepType-instant.nc')
    df = pd.DataFrame(index=ds['valid_time'], data=ds['t2m'][:, 0, 0])
    df.index = pd.to_datetime(df.index)
    df_list.append(df)

df = pd.concat(df_list)
df.columns = ['Observed']

# convert Kelvin to Celsius
df.Observed = df.Observed - 273.15
df_mst = df.copy()
df_mst.index = df_mst.index.shift(-7, freq='h')
df_mst = df_mst.iloc[7:]

# Calculate the slope of the temperature trend across yearly averages
df_yearly = df_mst.resample('YS').mean()
slope = np.polyfit(df_yearly.index.year, df_yearly['Observed'], 1)[0]
# Add climate adjusted data to dataframe
df_mst['Climate Adjusted'] = df_mst['Observed'] - slope*(df_mst.index.year-2020)

df_yearly = df_mst.resample('YS').mean()
df_yearly.plot()
plt.title('Annual Average Temperature in Phoenix')
plt.ylabel('Temperature (°C)')
plt.xlabel('Year')
plt.gca().get_lines()[1].set_linestyle('--')
plt.gca().get_lines()[0].set_color('orange')
plt.gca().get_lines()[1].set_color('red')
plt.legend()
plt.savefig('images/long_term_trend.png', dpi=300)
plt.close()

# Save the processed data
df_mst.to_csv('data/processed/observed_time_series.csv')

offset = round(df_mst['Climate Adjusted'].mean(), 8)
print('offset: ', offset)
scale = round(df_mst['Climate Adjusted'].std(), 8)
print('scale:  ', scale)
dft = (df_mst['Climate Adjusted']-offset)/scale

data_params = pd.Series({'offset': offset, 'scale': scale}, name='values')
data_params.to_csv('data/data_params.csv')

values = dft.values.squeeze()  # (T,) shape if single column
timestamps = dft.index

# Parameters
window_size = INPUT_SIZE  # 64 days
hop_size = int(window_size/4)     # 16 days
total_hours = len(values)

# Generate overlapping windows
X = []
index = []

while True:
    # Random phase shift between -12 and +12 hours
    phase_shift = np.random.randint(-12, 13) # replace with phase_shift = 0
    start = len(X) * hop_size
    shifted_start = start + phase_shift
    # Ensure shifted_start is within valid range
    shifted_start = max(0, min(shifted_start, total_hours - window_size))
    end = shifted_start + window_size
    if end > total_hours:
        break
    X.append(values[shifted_start:end])
    index.append(timestamps[shifted_start])
    if end == total_hours:
        break

X = np.array(X)  # Shape: (num_chunks, window_size)
index = pd.to_datetime(index)

# Create DataFrame with timestamp index
dft_reshaped = pd.DataFrame(data=X, index=index)

# Keep only rows where the index month is January
# dft_reshaped = dft_reshaped[dft_reshaped.index.month.isin([1])]

# Shuffle rows reproducibly
dft_reshaped = dft_reshaped.sample(frac=1)

# Save to CSV
dft_reshaped.to_csv('data/processed/phoenix_64days.csv')

<IPython.core.display.Javascript object>

offset:  23.27571895
scale:   9.89200131


## Model

In [13]:
# import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Build the encoder
def build_encoder():
    inputs = layers.Input(shape=(input_shape,))
    x = layers.Reshape((-1, 1))(inputs)
    x = layers.Conv1D(interim_filters, 5, strides=3, padding='same', activation='relu')(x)
    x = layers.Conv1D(interim_filters, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv1D(interim_filters, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv1D(interim_filters, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv1D(interim_filters, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv1D(2*latent_filter, 3, strides=2, padding='same')(x)
    z_mean = x[: ,:, :latent_filter]
    z_log_var = x[:, :, latent_filter:]
    z = Sampling()([z_mean, z_log_var])
    encoder = models.Model(inputs, [z_mean, z_log_var, z], name='encoder')
    return encoder

# Build the decoder
def build_decoder():
    latent_inputs = layers.Input(shape=(latent_dim, latent_filter))
    x = layers.Conv1DTranspose(interim_filters, 3, strides=2, padding='same', activation='relu')(latent_inputs)
    x = layers.Conv1DTranspose(interim_filters, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv1DTranspose(interim_filters, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv1DTranspose(interim_filters, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv1DTranspose(interim_filters, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv1DTranspose(1, 5, strides=3, padding='same')(x)
    outputs = layers.Reshape((-1,))(x)
    decoder = models.Model(latent_inputs, outputs, name='decoder')
    return decoder

def build_seasonal_prior():
    seasonal_inputs = layers.Input(shape=(None, 2*DEGREE,))
    x = layers.Dense(2*latent_filter, use_bias=False)(seasonal_inputs)
    z_mean = x[:, :, :latent_filter]
    z_log_var = x[:, :, latent_filter:]
    z = Sampling()([z_mean, z_log_var])
    seasonal_prior = models.Model(seasonal_inputs, [z_mean, z_log_var, z], name='seasonal_prior')
    return seasonal_prior

class VAE(models.Model):
    def __init__(self, encoder, decoder, prior, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.prior = prior # NOTE COMMENT THIS LINE OUT IF NO SEASONAL PRIOR IS USED
        self.noise_log_var = self.add_weight(name='var', shape=(1,), initializer='zeros', trainable=True)

    @tf.function
    def vae_loss(self, data):
        values = data[0]
        seasonal = data[1]
        z_mean, z_log_var, z = self.encoder(values)
        reconstructed = self.decoder(z)
        reconstruction_loss = -log_lik_normal_sum(values, reconstructed, self.noise_log_var)/INPUT_SIZE
        seasonal_z_mean, seasonal_z_log_var, _ = self.prior(seasonal) # NOTE COMMENT THIS LINE OUT IF NO SEASONAL PRIOR IS USED
        kl_loss_z = kl_divergence_sum(z_mean, z_log_var, seasonal_z_mean, seasonal_z_log_var)/INPUT_SIZE # NOTE COMMENT THIS LINE OUT IF NO SEASONAL PRIOR IS USED
        kl_loss_z = kl_divergence_standard_normal(z_mean, z_log_var) / INPUT_SIZE # Standard normal prior: mean=0, log_var=0 (=> std=1)
        return reconstruction_loss, kl_loss_z

    def train_step(self, data):
        with tf.GradientTape() as tape:
            reconstruction_loss, kl_loss_z = self.vae_loss(data)
            total_loss = reconstruction_loss + kl_loss_z
        
        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        return {'loss': total_loss}
    
    def test_step(self, data):
        reconstruction_loss, kl_loss_z = self.vae_loss(data)

        return {'loss': reconstruction_loss + kl_loss_z, 'recon': reconstruction_loss, 'kl': kl_loss_z}

    def call(self, inputs):
        _, _, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        return reconstructed


## Training

In [14]:
os.makedirs('model', exist_ok=True)
data = pd.read_csv('data/processed/phoenix_64days.csv', index_col=0, parse_dates=True)

fourier = lambda x: np.stack(
    [np.sin(2*np.pi*i*x) for i in range(1, DEGREE+1)] + 
    [np.cos(2*np.pi*i*x) for i in range(1, DEGREE+1)], axis=-1)

starting_day = np.array(data.index.dayofyear)[:, np.newaxis] - 1
data_days = (starting_day + np.arange(0, INPUT_SIZE//24, LATENT_SIZE//24))%365
seasonal_data = fourier(data_days/365)

training_ratio = 0.8

train = data.values[:int(len(data)*training_ratio)]
test = data.values[int(len(data)*training_ratio):]
train_seasonal = seasonal_data[:int(len(data)*training_ratio)]
test_seasonal = seasonal_data[int(len(data)*training_ratio):]

# convert to tensors
train_tensor = tf.convert_to_tensor(train, dtype=tf.float32)
test_tensor = tf.convert_to_tensor(test, dtype=tf.float32)
train_seasonal_tensor = tf.convert_to_tensor(train_seasonal, dtype=tf.float32)
test_seasonal_tensor = tf.convert_to_tensor(test_seasonal, dtype=tf.float32)

encoder = build_encoder()
encoder.summary()
decoder = build_decoder()
decoder.summary()
seasonal_prior = build_seasonal_prior()
seasonal_prior.summary()

# Create the VAE model
vae = VAE(encoder=encoder, decoder=decoder, prior=seasonal_prior)
optimizer = Adam(learning_rate=learning_rate)
vae.compile(optimizer=optimizer)

history = vae.fit(
    train_tensor, train_seasonal_tensor, 
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(test_tensor, test_seasonal_tensor),)

# Save model weights
_ = vae(train_tensor[:1])  # Call model once to build it
vae.save_weights('model/model_weights.weights.h5')

# Model reconstruction accuracy
print('noise emission sigma: ', np.exp(0.5*vae.noise_log_var)[0])

# Loss on validation data
recon_loss, kl_loss = vae.vae_loss((test_tensor, test_seasonal_tensor))
recon_loss = recon_loss.numpy()
kl_loss = kl_loss.numpy()
total_loss = recon_loss + kl_loss

# Save loss information as a dataframe
summary_df = pd.DataFrame({
    'Reconstruction loss': [recon_loss],
    'KL loss': [kl_loss],
    'Total loss': [total_loss]
})
summary_df.to_csv('data/train_summary.csv', index=False)

print('Reconstruction loss: ', recon_loss)
print('KL loss:             ', kl_loss)
print('Total loss:          ', total_loss)


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 1536)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_6 (Reshape) │ (None, 1536, 1)   │          0 │ input_layer_9[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_18 (Conv1D)  │ (None, 512, 20)   │        120 │ reshape_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_19 (Conv1D)  │ (None, 256, 20)   │      1,220 │ conv1d_18[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_20 (Conv1D)  │ (None, 128, 20)   │      1,220 │ conv1d_19[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_21 (Conv1D)  │ (None, 64, 20)    │      1,220 │ conv1d_20[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_22 (Conv1D)  │ (None, 32, 20)    │      1,220 │ conv1d_21[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_23 (Conv1D)  │ (None, 16, 40)    │      2,440 │ conv1d_22[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_12         │ (None, 16, 20)    │          0 │ conv1d_23[0][0]   │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_13         │ (None, 16, 20)    │          0 │ conv1d_23[0][0]   │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sampling_6          │ (None, 16, 20)    │          0 │ get_item_12[0][0… │
│ (Sampling)          │                   │            │ get_item_13[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,440 (29.06 KB)

 Trainable params: 7,440 (29.06 KB)

 Non-trainable params: 0 (0.00 B)

...

 Trainable params: 240 (960.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500


C:\Users\lokes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\optimizers\base_optimizer.py:731: UserWarning: Gradients do not exist for variables ['kernel'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 1.3804 - val_loss: 1.4866 - val_recon: 1.4866 - val_kl: 1.4182e-06
Epoch 2/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 1.3797 - val_loss: 1.4854 - val_recon: 1.4854 - val_kl: 8.0482e-08
Epoch 3/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.3795 - val_loss: 1.4846 - val_recon: 1.4846 - val_kl: 1.3756e-08
Epoch 4/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 1.3793 - val_loss: 1.4844 - val_recon: 1.4844 - val_kl: 3.2208e-09
Epoch 5/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 1.3791 - val_loss: 1.4844 - val_recon: 1.4844 - val_kl: 3.7738e-08
Epoch 6/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.3790 - val_loss: 1.4847 - val_recon: 1.4847 - val_kl: 9.3365e-08
Epoch 7/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 1.3790 - val_loss: 1.4846 - val_recon: 1.4846 - val_kl: 1.0501e-07
Epoch 8/500
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 1.3790 - val_loss: 1.4847 - val_recon: 1.4847 - val_kl: 1.4959e

## Generating new data

In [15]:
%matplotlib notebook

os.makedirs('data/processed', exist_ok=True)
data = pd.read_csv('data/processed/phoenix_64days.csv', index_col=0, parse_dates=True)

# To demonstrate loading, create a new instance and load weights
encoder = build_encoder()
decoder = build_decoder()
seasonal_prior = build_seasonal_prior()
vae = VAE(encoder=encoder, decoder=decoder, prior=seasonal_prior)
vae.compile()

starting_day = np.array(data.index.dayofyear)[:, np.newaxis] - 1
data_days = (starting_day + np.arange(0, INPUT_SIZE//24, LATENT_SIZE//24))%365
seasonal_data = fourier(data_days/365)

gen_dataset = data.values
gen_dataset_seasonal = seasonal_data

# convert to tensors
gen_dataset_tensor = tf.convert_to_tensor(gen_dataset, dtype=tf.float32)
gen_dataset_seasonal_tensor = tf.convert_to_tensor(gen_dataset_seasonal, dtype=tf.float32)

# Call the model once to build it
dummy_input = tf.zeros((1, gen_dataset.shape[1]), dtype=tf.float32)  # shape (1, 1536)
_ = vae(dummy_input)
vae.load_weights('model/model_weights.weights.h5')

encoded_mean, encoded_log_var, encoded_z = encoder(gen_dataset_tensor)

# Save latent vectors
latent_vectors = encoded_mean.numpy()  # Shape: (num_samples, latent_dim, latent_filter)
flat_latents = latent_vectors.reshape(latent_vectors.shape[0], -1)  # Flatten for saving

# Save as CSV
latent_df = pd.DataFrame(flat_latents)
latent_df.index = data.index[:len(latent_df)]  # Optional: align with input time series
latent_df.to_csv("data/processed/latent_vectors.csv")

# Save as .npy for quick load
np.save("data/processed/latent_vectors.npy", latent_vectors)

# set figure size
plt.figure(figsize=(15, 5))
# boxplot of encoded_mean
plt.subplot(1, 2, 1)
plt.boxplot(encoded_mean.numpy().reshape(-1, latent_filter))
plt.title('Encoded Mean')

# boxplot of encoded_log_var
plt.subplot(1, 2, 2)
plt.boxplot(encoded_log_var.numpy().reshape(-1, latent_filter))
plt.title('Encoded Log Variance')

plt.savefig("images/encoded_log_variance.png", dpi=300)

start_date = '1970-01-01 00:00:00'
end_date = '2020-12-31 16:00:00' 
dt = pd.date_range(start=start_date, end=end_date, freq='h')

gen_seasonal_inputs = fourier((dt.dayofyear[::LATENT_SIZE])/365)[np.newaxis]
_, _, z_gen = seasonal_prior(gen_seasonal_inputs)

gen_mean = decoder(z_gen).numpy()
noise = np.random.normal(size=gen_mean.shape)*np.exp(0.5*vae.noise_log_var[0].numpy())
gen = gen_mean + noise
n_latents = int(np.ceil(len(dt) / LATENT_SIZE))
z_gen = np.random.normal(size=(1, n_latents, latent_filter)).astype(np.float32)

gen_mean = decoder(z_gen).numpy()

data_params = pd.read_csv('data/data_params.csv', index_col=0)
offset = data_params.loc['offset', 'values']
scale = data_params.loc['scale', 'values']

gen_series = pd.Series(gen[0, :len(dt)]*scale + offset, index=dt)
gen_series = gen_series.rename('temperature')
gen_series = gen_series.to_frame()
gen_series.index.name = 'time'

# Add back seasonal cycle
df_obs = pd.read_csv('data/processed/observed_time_series.csv', index_col=0, parse_dates=True)
df_obs['weekofyear'] = df_obs.index.isocalendar().week
weekly_climatology = df_obs.groupby('weekofyear')['Observed'].mean()

# Map each hour in generated time series to its weekly mean
week_numbers = gen_series.index.isocalendar().week
seasonal_cycle = np.array([weekly_climatology[w] for w in week_numbers])
gen_series['temperature'] += seasonal_cycle

gen_series.to_csv('data/processed/generated.csv')

print(f"Succesfully generated time series of length: {len(gen_series.iloc[:,0])}")

gen_series['temperature'].resample('M').mean().plot(title='Monthly Mean of Generated Series')
plt.savefig('images/generated_monthly_avg.png', dpi=300)

C:\Users\lokes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


<IPython.core.display.Javascript object>

Succesfully generated time series of length: 447065


C:\Users\lokes\AppData\Local\Temp\ipykernel_32200\1397737070.py:95: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  gen_series['temperature'].resample('M').mean().plot(title='Monthly Mean of Generated Series')


### Latent Space Analysis

In [16]:
data = pd.read_csv('data/processed/phoenix_64days.csv', index_col=0, parse_dates=True)

fourier = lambda x: np.stack(
    [np.sin(2*np.pi*i*x) for i in range(1, DEGREE+1)] + 
    [np.cos(2*np.pi*i*x) for i in range(1, DEGREE+1)], axis=-1)

# To demonstrate loading, create a new instance and load weights
encoder = build_encoder()
decoder = build_decoder()
seasonal_prior = build_seasonal_prior()
vae = VAE(encoder=encoder, decoder=decoder, prior=seasonal_prior)
# optimizer = Adam(learning_rate=learning_rate)
vae.compile()

starting_day = np.array(data.index.dayofyear)[:, np.newaxis] - 1
data_days = (starting_day + np.arange(0, INPUT_SIZE//24, LATENT_SIZE//24))%365
seasonal_data = fourier(data_days/365)

#------------------------------------------------------------------------------
gen_dataset = data.values
gen_dataset_seasonal = seasonal_data

gen_dataset_tensor = tf.convert_to_tensor(gen_dataset, dtype=tf.float32) # convert to tensors
gen_dataset_seasonal_tensor = tf.convert_to_tensor(gen_dataset_seasonal, dtype=tf.float32)
#------------------------------------------------------------------------------

# Call the model once to build it
dummy_input = tf.zeros((1, gen_dataset.shape[1]), dtype=tf.float32)  # shape (1, 1536)
_ = vae(dummy_input)
vae.load_weights('model/model_weights.weights.h5')

encoded_mean, encoded_log_var, encoded_z = encoder(gen_dataset_tensor)

# Save latent vectors
latent_vectors = encoded_mean.numpy()  # Shape: (num_samples, latent_dim, latent_filter)
flat_latents = latent_vectors.reshape(latent_vectors.shape[0], -1)  # Flatten for saving

# Save as CSV
latent_df = pd.DataFrame(flat_latents)
latent_df.index = data.index[:len(latent_df)]  # Optional: align with input time series
latent_df.to_csv("data/processed/latent_vectors.csv")

# Save as .npy for quick load
np.save("data/processed/latent_vectors.npy", latent_vectors)

C:\Users\lokes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## Latent Space Plotting Stuff

In [17]:
%matplotlib notebook

# Setup
os.makedirs('images', exist_ok=True)

# Load data
latent_vectors = np.load("data/processed/latent_vectors.npy")
data = pd.read_csv('data/processed/phoenix_64days.csv', index_col=0, parse_dates=True)

if latent_vectors.shape[0] != data.shape[0]:
    print(f"Mismatch: {latent_vectors.shape[0]} latent vectors vs {data.shape[0]} rows in CSV. Trimming data.")
    data = data.iloc[:latent_vectors.shape[0]]

# Colour labels
mean_temps = data.mean(axis=1)
day_of_year_frac = data.index.dayofyear / 365.0
hour_of_day_frac = data.index.hour / 24.0

# Flatten latent vectors
flat_latents = latent_vectors.reshape(latent_vectors.shape[0], -1)

# Reduce dimensions
print("First performing a PCA")
pca = PCA(n_components=2)
pca_latents = pca.fit_transform(flat_latents)
print("PCA done")

print("Now performing t-SNE")
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
tsne_latents = tsne.fit_transform(flat_latents)
print("t-SNE done")

print("Now performing UMAP")
umap_model = umap.UMAP(n_components=2, random_state=42)
umap_latents = umap_model.fit_transform(flat_latents)
print("UMAP done")

# Helper for plotting
colorings = {
    "Mean Temp": mean_temps,
    "Day of Year (Fraction)": day_of_year_frac,
    "Hour of Day (Fraction)": hour_of_day_frac
}

reduced_spaces = {
    "Raw": flat_latents,
    "PCA": pca_latents,
    "t-SNE": tsne_latents,
    "UMAP": umap_latents
}

# Plot
fig, axs = plt.subplots(4, 3, figsize=(18, 20))

# Define colormaps for each coloring
cmap_dict = {
    "Mean Temp": "coolwarm",
    "Day of Year (Fraction)": "viridis",
    "Hour of Day (Fraction)": "plasma"
}

for i, (method_name, embedding) in enumerate(reduced_spaces.items()):
    for j, (label_name, label_values) in enumerate(colorings.items()):
        print(f"Plotting {method_name} coloured by {label_name}")
        ax = axs[i, j]
        cmap = cmap_dict.get(label_name, "viridis")
        scatter = ax.scatter(embedding[:, 0], embedding[:, 1], c=label_values, cmap=cmap, s=8, alpha=0.7)
        ax.set_title(f"{method_name} - Coloured by {label_name}")
        ax.set_xticks([])
        ax.set_yticks([])
        fig.colorbar(scatter, ax=ax, fraction=0.03, pad=0.01)

plt.tight_layout()
plt.savefig("images/latent_space_analysis.png", dpi=500)
plt.close()
print("Saved mega plot to images/latent_space_analysis.png")


First performing a PCA
PCA done
Now performing t-SNE
t-SNE done
Now performing UMAP


C:\Users\lokes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP done


<IPython.core.display.Javascript object>

Plotting Raw coloured by Mean Temp
Plotting Raw coloured by Day of Year (Fraction)
Plotting Raw coloured by Hour of Day (Fraction)
Plotting PCA coloured by Mean Temp
Plotting PCA coloured by Day of Year (Fraction)
Plotting PCA coloured by Hour of Day (Fraction)
Plotting t-SNE coloured by Mean Temp
Plotting t-SNE coloured by Day of Year (Fraction)
Plotting t-SNE coloured by Hour of Day (Fraction)
Plotting UMAP coloured by Mean Temp
Plotting UMAP coloured by Day of Year (Fraction)
Plotting UMAP coloured by Hour of Day (Fraction)
Saved mega plot to images/latent_space_analysis.png


## Wrap up

In [18]:
# Define paths
folders_to_move = [
    'images',
    'model',
    os.path.join('data', 'processed')
]

files_to_move = [
    os.path.join('data', 'data_params.csv'),
    os.path.join('data', 'train_summary.csv')
]

# Create the results directory if it doesn't exist
os.makedirs(results_directory, exist_ok=True)

# Move folders
for folder in folders_to_move:
    if os.path.isdir(folder):
        dest_folder = os.path.join(results_directory, os.path.basename(folder))
        shutil.move(folder, dest_folder)
        print(f"Moved folder '{folder}' to '{dest_folder}'")
    else:
        print(f"Folder not found: {folder}")

# Move files
for file in files_to_move:
    if os.path.isfile(file):
        dest_file = os.path.join(results_directory, os.path.basename(file))
        shutil.move(file, dest_file)
        print(f"Moved file '{file}' to '{dest_file}'")
    else:
        print(f"File not found: {file}")

# Delete __pycache__ folder if it exists
pycache_folder = '__pycache__'
if os.path.isdir(pycache_folder):
    shutil.rmtree(pycache_folder)
    print(f"Deleted folder '{pycache_folder}'")

Moved folder 'images' to 'results/training_run_Seasonal_prior_True_phase_shift_500_epochs\images'
Moved folder 'model' to 'results/training_run_Seasonal_prior_True_phase_shift_500_epochs\model'
Moved folder 'data\processed' to 'results/training_run_Seasonal_prior_True_phase_shift_500_epochs\processed'
Moved file 'data\data_params.csv' to 'results/training_run_Seasonal_prior_True_phase_shift_500_epochs\data_params.csv'
Moved file 'data\train_summary.csv' to 'results/training_run_Seasonal_prior_True_phase_shift_500_epochs\train_summary.csv'
